In [44]:
from cvrp_algorithms import SimpleAco, AcoSigmaBestAnts, AcoImprovedPheromoneUpdate, Aco2opt, Greedy
import cvrpcases as cvrp
from array import *
import math
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.backends.backend_pdf import PdfPages

def listToString(list):
    str1 = ''   
    for ele in list:  
        str1 += str(ele)+'\n' 
    str1 = str1[:-2] 
    return str1  

def saveFile(firstLine, vector, file):
    text_file = open(file, "w+")
    text_file.write(firstLine+'\n')
    n = text_file.write(listToString(vector))
    text_file.close()

def readVector(file):
    file1 = open(file, 'r') 
    Lines = file1.readlines() 
    v = np.zeros(len(Lines)-1)
    for it in range(len(v)):
        v[it] = float(Lines[it+1])
    return v

def binarySearchAntsCount(refTime,iterations,case, mincount = 1,maxcount = 100):
    print("mincount =" + str(mincount))
    print("maxcount =" + str(maxcount))
    print("reftime =" + str(refTime))
    print("iterations =" + str(iterations))
    if(mincount == maxcount):
        algorithm = SimpleAco(cvrp.load_case(case), mincount, max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=10, seed = 78)
        start = time.time()
        solution, path = algorithm.compute(log_level=0)
        tim = time.time()-start
        print("retuur time="+str(tim)+" mincount=" + str(mincount))
        return tim, solution, mincount
    if(maxcount - mincount == 1):
        algorithm1 = SimpleAco(cvrp.load_case(case), mincount, max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=10, seed = 78)
        start1 = time.time()
        solution1, path1 = algorithm1.compute(log_level=0)
        end1 = time.time()
        algorithm2 = SimpleAco(cvrp.load_case(case), maxcount, max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=10, seed = 78)
        start2 = time.time()
        solution2, path2 = algorithm1.compute(log_level=0)
        end2 = time.time()
        if(abs(refTime - end1 + start1)<=abs(refTime - end2 + start2)):
            print("retuur time="+str(end1-start1)+" mincount=" + str(mincount))
            return end1-start1, solution1, mincount
        print(end1-start1)
        print("**********")
        return end2-start2, solution2, maxcount
    algorithm = SimpleAco(cvrp.load_case(case), math.floor((maxcount + mincount)/2), max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=10, seed = 78)
    start = time.time()
    solution, path = algorithm.compute(log_level=0)
    time1 = time.time() - start
    if(time1>=refTime):
        return binarySearchAntsCount(refTime,iterations,case,mincount,math.floor((maxcount + mincount)/2))
    print("Im here")
    return binarySearchAntsCount(refTime,iterations,case,math.floor((maxcount + mincount)/2),maxcount)

In [2]:
cases =[
r'datasets\A\A-n32-k5.vrp']

print('działa')

res = np.zeros((8,3,10))
times = np.zeros((8,3,10))
ants = np.zeros((8,3,10))
for case in range(1):
    for seed in range(3):
        start = time.time()
        algorithm = SimpleAco(cvrp.load_case(cases[case]), ants_count=1000, max_iterations=1, alpha=2, beta=5, evaporation_rate=1, pheromone_amount=10, seed = seed)
        res[0][seed][case], path = algorithm.compute(log_level=0)
        end = time.time()
        times[0][seed][case] = end - start
        ants[0][seed][case] = 1000
        for it in range(0,7):
            print("*************it = "+str(it)+" seed = "+ str(seed) + " case = " + str(case))
            times[it+1][seed][case], res[it+1][seed][case], ants[it+1][seed][case] = binarySearchAntsCount(times[0][seed][case],it+2,cases[case],1,ants[it][seed][case])
            

saveFile('res',res,r'results/h3/res.txt')
saveFile('ants',ants,r'results/h3/ants.txt')
saveFile('times',times,r'results/h3/times.txt')

działa
*************it = 0 seed = 0 case = 0
mincount =1
maxcount =1000.0
reftime =3.3715322017669678
iterations =2
mincount =1
maxcount =500
reftime =3.3715322017669678
iterations =2
Im here
mincount =250
maxcount =500
reftime =3.3715322017669678
iterations =2
Im here
mincount =375
maxcount =500
reftime =3.3715322017669678
iterations =2
mincount =375
maxcount =437
reftime =3.3715322017669678
iterations =2
mincount =375
maxcount =406
reftime =3.3715322017669678
iterations =2
mincount =375
maxcount =390
reftime =3.3715322017669678
iterations =2
mincount =375
maxcount =382
reftime =3.3715322017669678
iterations =2
mincount =375
maxcount =378
reftime =3.3715322017669678
iterations =2
mincount =375
maxcount =376
reftime =3.3715322017669678
iterations =2
retuur time=3.4661149978637695 mincount=375
*************it = 1 seed = 0 case = 0
mincount =1
maxcount =375.0
reftime =3.3715322017669678
iterations =3
Im here
mincount =188
maxcount =375.0
reftime =3.3715322017669678
iterations =3
mincount

In [7]:
print(res[:,0,0])
print(res[:,1,0])
print(res[:,2,0])
print(ants[:,0,0])
print(times[:,0,0])




[1215.97881378  937.57432277  880.75260883  921.31689372  931.7776377
  998.83803038  974.08805281  904.20213686]
[1242.0909006   859.70049877  848.24873097  876.24658711  851.55161823
  869.32919758  932.86586277  930.99198844]
[1282.20320438  858.29524492  853.70526811  868.62193964  874.40018033
  837.53995413  869.88456016  910.85629112]
[1000.  375.  211.  161.  141.  106.   86.   75.]
[3.3715322  3.466115   3.16681981 3.23524761 3.36498237 3.23840308
 3.41314912 3.2206986 ]


In [37]:
mean1 = np.zeros((5,8))
sd1 = np.zeros((5,8))
for i in range(5):
    for j in range(8):
        mean1[i,j]= np.mean(res[j,:,i])
        sd1[i,j] = np.std(res[j,:,i])

meant = np.zeros((5,8))
sdt = np.zeros((5,8))
for i in range(5):
    for j in range(8):
        meant[i,j]= np.mean(times[j,:,i])
        sdt[i,j] = np.std(times[j,:,i])

meana = np.zeros((5,8))
sda = np.zeros((5,8))
for i in range(5):
    for j in range(8):
        meana[i,j]= np.mean(ants[j,:,i])
        sda[i,j] = np.std(ants[j,:,i])

In [8]:
cases =[
r'datasets\A\A-n32-k5.vrp',
r'datasets\A\A-n33-k5.vrp',
r'datasets\A\A-n33-k6.vrp',
r'datasets\A\A-n34-k5.vrp',
r'datasets\A\A-n36-k5.vrp']

print('działa')

for case in range(1,5):
    for seed in range(3):
        start = time.time()
        algorithm = SimpleAco(cvrp.load_case(cases[case]), ants_count=1000, max_iterations=1, alpha=2, beta=5, evaporation_rate=1, pheromone_amount=10, seed = seed)
        res[0][seed][case], path = algorithm.compute(log_level=0)
        end = time.time()
        times[0][seed][case] = end - start
        ants[0][seed][case] = 1000
        for it in range(0,7):
            print("*************it = "+str(it)+" seed = "+ str(seed) + " case = " + str(case))
            times[it+1][seed][case], res[it+1][seed][case], ants[it+1][seed][case] = binarySearchAntsCount(times[0][seed][case],it+2,cases[case],1,ants[it][seed][case])
            

saveFile('res',res,r'results/h3/res.txt')
saveFile('ants',ants,r'results/h3/ants.txt')
saveFile('times',times,r'results/h3/times.txt')

incount =312
maxcount =375
reftime =4.482706308364868
iterations =3
mincount =312
maxcount =343
reftime =4.482706308364868
iterations =3
mincount =312
maxcount =327
reftime =4.482706308364868
iterations =3
Im here
mincount =319
maxcount =327
reftime =4.482706308364868
iterations =3
Im here
mincount =323
maxcount =327
reftime =4.482706308364868
iterations =3
mincount =323
maxcount =325
reftime =4.482706308364868
iterations =3
Im here
mincount =324
maxcount =325
reftime =4.482706308364868
iterations =3
retuur time=4.4051268100738525 mincount=324
*************it = 2 seed = 2 case = 3
mincount =1
maxcount =324.0
reftime =4.482706308364868
iterations =4
Im here
mincount =162
maxcount =324.0
reftime =4.482706308364868
iterations =4
mincount =162
maxcount =243
reftime =4.482706308364868
iterations =4
mincount =162
maxcount =202
reftime =4.482706308364868
iterations =4
Im here
mincount =182
maxcount =202
reftime =4.482706308364868
iterations =4
Im here
mincount =192
maxcount =202
reftime =4.48

In [50]:
with PdfPages(r'charts\hyp3_1.pdf') as export_pdf:
    plt.figure(figsize=(25,25))
    plt.plot(range(1,9), mean1[0,:], 'ro', label = "Przypadek [1]")
    plt.errorbar(range(1,9), mean1[0,:], yerr=sd1[0,:],color = 'r', ecolor = 'r')
    plt.plot(range(1,9), mean1[1,:], 'go', label = "Przypadek [2]")
    plt.errorbar(range(1,9), mean1[1,:], yerr=sd1[1,:],color = 'g', ecolor = 'g')
    plt.plot(range(1,9), mean1[2,:], 'bo', label = "Przypadek [3]")
    plt.errorbar(range(1,9), mean1[2,:], yerr=sd1[2,:],color = 'b', ecolor = 'b')
    plt.plot(range(1,9), mean1[3,:], 'yo', label = "Przypadek [4]")
    plt.errorbar(range(1,9), mean1[3,:], yerr=sd1[3,:],color = 'y', ecolor = 'y')
    plt.plot(range(1,9), mean1[4,:], 'mo', label = "Przypadek [5]")
    plt.errorbar(range(1,9), mean1[4,:], yerr=sd1[4,:],color = 'm', ecolor = 'm')
    plt.tick_params(axis='x', labelsize=30)
    plt.tick_params(axis='y', labelsize=30)
    plt.xlabel('Liczba iteracji głównej pętli', fontsize=30)
    plt.ylabel('wynik', fontsize=30)
    plt.legend(loc='upper right',prop={'size': 25})
    plt.title('Wynik działania algorytmu dla 5 przypadków testowych manewrując parametrami N i M', fontsize=35, pad=30)
    plt.grid(True, linestyle='-.')
    export_pdf.savefig()
    plt.close()

In [46]:
with PdfPages(r'charts\hyp3_2.pdf') as export_pdf:
    plt.figure(figsize=(25,25))
    plt.plot(range(1,9), meant[0,:], 'ro', label = "Przypadek [1]")
    plt.plot(range(1,9), meant[0,:], color = 'r')
    #plt.errorbar(range(1,9), meant[0,:], yerr=sdt[0,:],color = 'r', ecolor = 'r')
    plt.plot(range(1,9), meant[1,:], 'go', label = "Przypadek [2]")
    plt.plot(range(1,9), meant[1,:], color = 'g')
    #plt.errorbar(range(1,9), meant[1,:], yerr=sdt[1,:],color = 'g', ecolor = 'g')
    plt.plot(range(1,9), meant[2,:], 'bo', label = "Przypadek [3]")
    plt.plot(range(1,9), meant[2,:], color = 'b')
    #plt.errorbar(range(1,9), meant[2,:], yerr=sdt[2,:],color = 'b', ecolor = 'b')
    plt.plot(range(1,9), meant[3,:], 'yo', label = "Przypadek [4]")
    plt.plot(range(1,9), meant[3,:], color = 'y')
    #plt.errorbar(range(1,9), meant[3,:], yerr=sdt[3,:],color = 'y', ecolor = 'y')
    plt.plot(range(1,9), meant[4,:], 'mo', label = "Przypadek [5]")
    plt.plot(range(1,9), meant[4,:], color = 'm')
    #plt.errorbar(range(1,9), meant[4,:], yerr=sdt[4,:],color = 'm', ecolor = 'm')
    plt.axis([0.5, 8.5, 0, 6])
    plt.tick_params(axis='x', labelsize=30)
    plt.tick_params(axis='y', labelsize=30)
    plt.xlabel('Liczba iteracji głównej pętli', fontsize=30)
    plt.ylabel('wynik', fontsize=30)
    plt.legend(loc='lower right',prop={'size': 25})
    plt.title('Czas trwania obliczeń dla pojedynczych konfiguracji', fontsize=35, pad=30)
    plt.grid(True, linestyle='-.')
    export_pdf.savefig()
    plt.close()

In [49]:
with PdfPages(r'charts\hyp3_3.pdf') as export_pdf:
    plt.figure(figsize=(25,25))
    plt.plot(range(1,9), meana[0,:], 'ro', label = "Przypadek [1]")
    plt.errorbar(range(1,9), meana[0,:], yerr=sda[0,:],color = 'r', ecolor = 'r')
    plt.plot(range(1,9), meana[1,:], 'go', label = "Przypadek [2]")
    plt.errorbar(range(1,9), meana[1,:], yerr=sda[1,:],color = 'g', ecolor = 'g')
    plt.plot(range(1,9), meana[2,:], 'bo', label = "Przypadek [3]")
    plt.errorbar(range(1,9), meana[2,:], yerr=sda[2,:],color = 'b', ecolor = 'b')
    plt.plot(range(1,9), meana[3,:], 'yo', label = "Przypadek [4]")
    plt.errorbar(range(1,9), meana[3,:], yerr=sda[3,:],color = 'y', ecolor = 'y')
    plt.plot(range(1,9), meana[4,:], 'mo', label = "Przypadek [5]")
    plt.errorbar(range(1,9), meana[4,:], yerr=sda[4,:],color = 'm', ecolor = 'm')
    plt.tick_params(axis='x', labelsize=30)
    plt.tick_params(axis='y', labelsize=30)
    plt.xlabel('Liczba iteracji', fontsize=30)
    plt.ylabel('Liczba mrówek', fontsize=30)
    plt.legend(loc='upper right',prop={'size': 25})
    plt.title('Wykres liczba mrówek otrzymanych w algorytmie od liczby iteracji', fontsize=35, pad=30)
    plt.grid(True, linestyle='-.')
    export_pdf.savefig()
    plt.close()